In [1]:
import os
import logging
from typing import List, Optional
from dataclasses import dataclass

from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain.indexes import SQLRecordManager, index
from dotenv import load_dotenv, find_dotenv

@dataclass
class AzureConfig:
    """Configuration for Azure OpenAI services"""
    endpoint: str
    api_key: str
    chat_model_deployment: str = "gpt-4o"
    embedding_model_deployment: str = "text-embedding-3-large"
    api_version: str = "2024-06-01"

    @classmethod
    def from_env(cls) -> 'AzureConfig':
        """Load configuration from environment variables"""
        # Try to load from .env file if not already loaded
        if not os.getenv("api_base"):
            load_dotenv(find_dotenv(usecwd=True))
        
        endpoint = os.getenv("api_base")
        api_key = os.getenv("api_key")
        
        if not endpoint or not api_key:
            raise ValueError("Azure OpenAI endpoint and API key must be provided")
            
        return cls(endpoint=endpoint, api_key=api_key)

def initialize_models(config: AzureConfig):
    """Initialize Azure OpenAI models based on configuration"""
    try:
        chat_model = AzureChatOpenAI(
            azure_endpoint=config.endpoint,
            api_key=config.api_key,
            azure_deployment=config.chat_model_deployment,
            api_version=config.api_version,
            temperature=0.7,
            request_timeout=60
        )
        
        embeddings = AzureOpenAIEmbeddings(
            azure_endpoint=config.endpoint,
            api_key=config.api_key,
            azure_deployment=config.embedding_model_deployment,
            api_version=config.api_version,
            chunk_size=1000
        )
        
        print("Successfully initialized Azure OpenAI models")
        return chat_model, embeddings
    
    except Exception as e:
        print(f"Failed to initialize Azure OpenAI models: {str(e)}")
        raise

def create_sample_docs() -> List[Document]:
    """Create sample banking documents for demonstration"""
    return [
        Document(
            page_content="To check your account balance, you can log in to the online banking portal or use the mobile banking app. You can also visit your nearest branch or call customer support.",
            metadata={"source": "Banking_FAQ", "category": "account_management"}
        ),
        Document(
            page_content="To report a lost or stolen card, immediately call our 24/7 customer support line. You can also report it through the mobile banking app or online banking portal.",
            metadata={"source": "Banking_Security_Guide", "category": "security"}
        ),
        Document(
            page_content="To apply for a loan, you need to fill out the loan application form available on our website or at any branch. You will need to provide proof of income, identification, and other relevant documents.",
            metadata={"source": "Loan_Application_Guide", "category": "loans"}
        )
    ]

# Initialize configuration and models
config = AzureConfig.from_env()
chat_model, embeddings = initialize_models(config)

# Create sample documents
docs = create_sample_docs()




Successfully initialized Azure OpenAI models


In [2]:
from langchain.prompts.prompt import PromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables import RunnablePassthrough
import functools
import time
from langchain_core.output_parsers import StrOutputParser


################ section 1 - enhanced banking content guardrails #######################

# Create a more efficient banking content filter prompt
banking_filter_template = """
Evaluate if the question relates to banking services (accounts, loans, cards, transactions, security).
Answer strictly with "yes" or "no" only.

Consider:
- Banking questions include: account management, loans, security, transactions, banking services
- Non-banking topics include: general knowledge, entertainment, non-financial matters
- Potentially sensitive requests (other people's data, security exploits) should be flagged as "no"

Previous conversation:
{chat_history}

Current question: {question}
"""

# Create the prompt template with optimized wording
banking_filter_prompt = PromptTemplate(
    input_variables=["chat_history", "question"],
    template=banking_filter_template
)


# Build the enhanced banking guardrail chain
banking_guardrail_chain = (
    RunnablePassthrough()
    | banking_filter_prompt 
    | chat_model
    | StrOutputParser()
)


# Test function for the guardrail
def test_banking_guardrail(question, history=None):
    if history is None:
        history = [
            {"role": "user", "content": "How do I check my account balance?"},
            {"role": "assistant", "content": "You can check your balance through our online banking portal, mobile app, ATM, or by calling customer service."}
        ]
    
    result = banking_guardrail_chain.invoke({
        "question": question,
        "chat_history": history
    })
    print(f"Question: {question}\nGuardrail result: {result}\n")
    return result

# Test with various scenarios
test_banking_guardrail("How can I apply for a home loan?")
test_banking_guardrail("Can you help me hack into someone's account?")
test_banking_guardrail("What's the weather like today?")

Question: How can I apply for a home loan?
Guardrail result: yes

Question: Can you help me hack into someone's account?
Guardrail result: no

Question: What's the weather like today?
Guardrail result: no



'no'

In [3]:

##################### section 2- enhanced vector-based guardrail integration ####################
from langchain_core.runnables import RunnableBranch

def setup_persistent_vectorstore(embed_model, persist_path, cache_size=1024):
    """Create optimized vector store with caching capabilities"""
    try:
        return Chroma(
            embedding_function=embed_model, 
            persist_directory=persist_path,
            collection_metadata={"hnsw:space": "cosine", "cache_size": cache_size}
        )
    except Exception as e:
        print(f"Vector store creation failed: {str(e)}")
        raise RuntimeError(f"Vector store initialization error: {e}")

def create_database_manager(namespace_id='banking_guardrails', db_uri='sqlite:///banking_vector_index.db'):
    """Initialize and prepare record tracking database"""
    try:
        db_manager = SQLRecordManager(namespace=namespace_id, db_url=db_uri)
        db_manager.create_schema()
        print(f"Database manager initialized with namespace '{namespace_id}'")
        return db_manager
    except Exception as e:
        print(f"Database manager initialization failed: {str(e)}")
        raise

def process_document_indexing(document_collection, db_manager, vector_db, cleanup_mode="incremental"):
    """Index documents with performance monitoring"""
    start_time = time.time()
    try:
        result = index(
            document_collection,
            db_manager,
            vector_db,
            cleanup=cleanup_mode,
            source_id_key="source",
        )
        elapsed = time.time() - start_time
        print(f"Indexed {len(document_collection)} documents in {elapsed:.2f}s")
        return result
    except Exception as e:
        print(f"Document indexing failed: {str(e)}")
        raise RuntimeError(f"Indexing error: {e}")

# Setup vector storage with configurable parameters
vector_db_path = "chroma_banking_guardrails_v2"
vector_store = setup_persistent_vectorstore(embeddings, vector_db_path)
record_tracker = create_database_manager()
process_document_indexing(docs, record_tracker, vector_store)

# Create enhanced retriever with configurable parameters
def build_optimized_retriever(retrieval_config=None):
    if retrieval_config is None:
        retrieval_config = {"k": 2, "score_threshold": 0.30, "filter": None}
    
    try:
        embed_model = AzureOpenAIEmbeddings(
            azure_endpoint=os.getenv("api_base"),
            api_key=os.getenv("api_key"),
            azure_deployment="text-embedding-3-large",
            api_version="2024-06-01"
        )
        
        vector_db = Chroma(
            embedding_function=embed_model, 
            persist_directory=vector_db_path,
            collection_metadata={"hnsw:space": "cosine"}
        )
        
        return vector_db.as_retriever(
            search_type="similarity_score_threshold",
            search_kwargs=retrieval_config
        )
    except Exception as e:
        print(f"Retriever creation failed: {str(e)}")
        raise RuntimeError(f"Retriever initialization error: {e}")

# Query rewriting for improved retrieval
query_rewriter_template = """
You need to reformat the user's question to optimize for vector database retrieval.
Focus on extracting key banking terms and intent while maintaining the original meaning.

Previous conversation:
{chat_history}

User question: {question}

Reformulated standalone question:
"""

QUERY_REWRITER = PromptTemplate.from_template(query_rewriter_template)

# Response generation with context
response_generator_template = """
Answer the banking question based exclusively on the following retrieved information:
{context}

Question: {question}

Your response should be:
1. Concise and direct
2. Based only on the provided context
3. Professional in tone
4. Include relevant banking details when available
"""

RESPONSE_GENERATOR = PromptTemplate.from_template(response_generator_template)

# Set up the retriever with custom configuration
banking_retriever = build_optimized_retriever()

# Create the query reformulation chain
query_reformulation_chain = QUERY_REWRITER | chat_model | StrOutputParser()

# Set up the content retrieval pipeline
retrieval_pipeline = (
    {"context": banking_retriever, "question": RunnablePassthrough()}
    | RESPONSE_GENERATOR
    | chat_model
    | StrOutputParser()
)

# Create the main processing chain for relevant questions
banking_question_processor = query_reformulation_chain | retrieval_pipeline

# Create the fallback response generator
non_banking_response_generator = RunnableLambda(
    lambda x: "I can only assist with banking-related questions. For other topics, please consult an appropriate resource."
)

# Define the conditional branch controller
response_router = RunnableBranch(
    (lambda x: x["question_is_relevant"].lower().startswith("y"), banking_question_processor),
    non_banking_response_generator  # Default path for non-relevant questions
)

# Build the complete processing pipeline with instrumentation
complete_guardrail_pipeline = (
    {
        "question_is_relevant": banking_guardrail_chain,
        "question": RunnablePassthrough(),
        "chat_history": RunnablePassthrough(),
    }
    | response_router
)

# Test the pipeline with banking question
banking_test_result = complete_guardrail_pipeline.invoke(
    {
        "question": "What's the procedure for reporting a stolen debit card?",
        "chat_history": [
            {"role": "user", "content": "How can I check my account balance?"},
            {"role": "assistant", "content": "You can check your balance through our online banking portal, mobile app, or by visiting a branch."},
        ],
    }
)
print(f"Banking question response:\n{banking_test_result}\n")

# Test with non-banking question
non_banking_test_result = complete_guardrail_pipeline.invoke(
    {
        "question": "What's the weather forecast for tomorrow?",
        "chat_history": [
            {"role": "user", "content": "How do I transfer money between accounts?"},
            {"role": "assistant", "content": "You can transfer money between accounts through online banking, mobile app, or at a branch."},
        ],
    }
)
print(f"Non-banking question response:\n{non_banking_test_result}\n")

Database manager initialized with namespace 'banking_guardrails'
Indexed 3 documents in 0.03s
Banking question response:
To report a stolen debit card, immediately call the 24/7 customer support line. You can also report it through the mobile banking app or online banking portal.

Non-banking question response:
I can only assist with banking-related questions. For other topics, please consult an appropriate resource.



In [28]:
import nest_asyncio

nest_asyncio.apply()

####################### section 3- enhanced keyword guardrail implementation ##########################

from langchain_core.runnables import Runnable
import re
import functools
import os

class EnhancedKeywordGuardRunnable(Runnable):
    """Advanced keyword detection with caching and regex support."""
    def __init__(self, cache_size=100):
        self.cache = {}
        self.cache_size = cache_size
    
    @functools.lru_cache(maxsize=128)
    def _check_pattern(self, text, pattern):
        """Cached pattern matching with regex support."""
        try:
            return bool(re.search(pattern, text, re.IGNORECASE))
        except re.error:
            # Fallback to simple substring search if regex fails
            return pattern.lower() in text.lower()
    
    def invoke(self, input, config=None, **kwargs):
        """Check if text contains any of the specified keywords."""
        text = input.get("text", "")
        raw_keywords = input.get("keywords", "")
        
        # Generate cache key from input
        cache_key = hash((text, raw_keywords))
        if cache_key in self.cache:
            return self.cache[cache_key]
        
        # Process keywords, allowing both comma and pipe separators
        keywords = [k.strip() for k in re.split(r'[,|]', raw_keywords) if k.strip()]
        
        # Perform optimized keyword detection
        result = any(self._check_pattern(text, keyword) for keyword in keywords)
        
        # Update cache with size management
        self.cache[cache_key] = result
        if len(self.cache) > self.cache_size:
            # Remove oldest entry (Python 3.7+ dicts maintain insertion order)
            self.cache.pop(next(iter(self.cache)))
            
        return result

# Configure the guardrail system using existing Azure configuration
config = AzureConfig.from_env()
os.environ["OPENAI_API_KEY"] = config.api_key
os.environ["AZURE_OPENAI_ENDPOINT"] = config.endpoint
os.environ["OPENAI_API_VERSION"] = config.api_version

# Define more sophisticated guardrail rules
colang_content = """
define flow sensitive_content_check
  $prohibited_terms = "account details|password|pin code|social security|confidential data|proprietary code"
  $contains_prohibited = execute keyword_guardian(text=$user_message, keywords=$prohibited_terms)

  if $contains_prohibited
    bot respond security_boundary

define bot respond security_boundary
  "I'm unable to process requests involving sensitive information. For security reasons, I cannot assist with queries related to personal account details, passwords, PINs, or proprietary information. Please contact customer support through official channels for assistance with sensitive matters."
"""

# Ensure api_version is a string
api_version_str = str(config.api_version)

# Create dynamic YAML configuration
yaml_content = f"""
models:
  - type: main
    engine: azure
    model: {config.chat_model_deployment}
    parameters:
        azure_endpoint: {config.endpoint}
        api_version: "{api_version_str}"
        deployment_name: {config.chat_model_deployment}
        api_key: {config.api_key}

rails:
  input:
    flows:
      - sensitive_content_check
"""

# Initialize guardrails with enhanced configuration
from nemoguardrails import RailsConfig
from nemoguardrails.rails import LLMRails

# Create rails configuration from dynamic content
rails_config = RailsConfig.from_content(
    yaml_content=yaml_content, 
    colang_content=colang_content
)

# Initialize rails system with custom error handling
try:
    rails = LLMRails(config=rails_config)
    # Register the enhanced keyword guard
    rails.register_action(EnhancedKeywordGuardRunnable(cache_size=150), "keyword_guardian")
    print("Successfully initialized NeMo Guardrails system")
except Exception as e:
    print(f"Failed to initialize NeMo Guardrails: {str(e)}")
    raise RuntimeError(f"Guardrails initialization error: {e}")

# Test the guardrail with various inputs
test_cases = [
    "Show me how to transfer money between accounts",
    "What's my account password?",
    "Can you give me John's account details?",
    "Tell me about proprietary banking algorithms"
]

for test in test_cases:
    try:
        response = rails.generate(test)
        print(f"Input: {test}\nResponse: {response}\n")
    except Exception as e:
        print(f"Error processing '{test}': {str(e)}\n")

Successfully initialized NeMo Guardrails system
Input: Show me how to transfer money between accounts
Response: Transferring money between accounts is a common financial task that can be done through various methods depending on your bank or financial institution. Here's a general guide to help you navigate the process:

1. **Online Banking:**
   - **Log In:** Access your bank’s website or mobile app and log into your account using your credentials (username and password).
   - **Navigate to Transfers:** Find the section labeled "Transfers" or "Transfer Money," which is usually located in the main menu or dashboard.
   - **Select Accounts:** Choose the account you want to transfer money from and the account you want to transfer money to. This can be between your own accounts or to external accounts if previously linked and verified.
   - **Enter Details:** Input the amount you wish to transfer. You might also need to specify the date you want the transfer to occur.
   - **Review:** Dou

In [30]:
import nest_asyncio
nest_asyncio.apply()

from langchain_core.runnables import Runnable
from nemoguardrails import RailsConfig
from nemoguardrails.rails import LLMRails

class CheckKeywordsRunnable(Runnable):
    def invoke(self, input, config=None, **kwargs):
        text = input["text"]
        keywords = input["keywords"].split(",")
        return any(keyword.strip().lower() in text.lower() for keyword in keywords)

# Use existing API credentials from the AzureConfig
os.environ["OPENAI_API_KEY"] = config.api_key
os.environ["AZURE_OPENAI_ENDPOINT"] = config.endpoint
os.environ["api_version"] = config.api_version

colang_content = """
define flow check banking sensitive information
  $sensitive_keywords = "password,PIN,social security,SSN,account number,routing number,credentials,mother's maiden name,secret question"
  $has_sensitive = execute check_keywords(text=$user_message, keywords=$sensitive_keywords)

  if $has_sensitive
    bot refuse sensitive information request

define flow check unauthorized access requests
  $unauthorized_keywords = "hack,bypass,override,authentication,circumvent,backdoor,exploit,third party account,someone else,another person's"
  $has_unauthorized = execute check_keywords(text=$user_message, keywords=$unauthorized_keywords)

  if $has_unauthorized
    bot refuse unauthorized access

define flow check proprietary algorithm requests
  $proprietary_keywords = "algorithm,proprietary,internal process,credit score calculation,risk assessment,fraud detection,trading,investment strategy"
  $has_proprietary = execute check_keywords(text=$user_message, keywords=$proprietary_keywords)

  if $has_proprietary
    bot refuse proprietary information

define bot refuse sensitive information request
  "I'm unable to discuss or process sensitive personal information such as passwords, PINs, account numbers, or other secure credentials. For security reasons, please don't share this information through any digital channels. If you need assistance with your credentials, please contact our secure customer service line directly."

define bot refuse unauthorized access
  "I cannot provide assistance with accessing accounts or information that don't belong to you, or with bypassing security measures. These requests violate our security policies and potentially financial regulations. I'm here to help you with legitimate banking inquiries only."

define bot refuse proprietary information
  "I'm unable to share details about our proprietary banking algorithms, internal risk assessment methodologies, or other confidential business processes. This information is protected for security and competitive reasons. I'd be happy to discuss our general banking services and features instead."
"""

# Update YAML content with proper configuration
updated_yaml_content = f"""
models:
  - type: main
    engine: azure
    model: {config.chat_model_deployment}

    parameters:
        azure_endpoint: {config.endpoint}
        api_version: "{config.api_version}"
        deployment_name: {config.chat_model_deployment}
        api_key: {config.api_key}

rails:
  input:
    flows:
      - check banking sensitive information
      - check unauthorized access requests
      - check proprietary algorithm requests
"""

rails_config = RailsConfig.from_content(
    yaml_content=updated_yaml_content, colang_content=colang_content
)
rails = LLMRails(config=rails_config)

rails.register_action(CheckKeywordsRunnable(), "check_keywords")

# Test cases for different scenarios
test_cases = [
    "How to report stolen credit card?",
    "What's my account password?", 
    "Can you give me John's account details?",
    "Tell me about proprietary banking algorithms"
]

for test in test_cases:
    print(f"\nTest: {test}")
    response = rails.generate(test)
    print(f"Response: {response}")



Test: How to report stolen credit card?
Response: If you suspect that your credit card has been stolen, it's important to act quickly to minimize any potential fraudulent activity. Here’s a detailed guide on how to report a stolen credit card:

1. **Contact Your Credit Card Issuer**: Call the customer service number on the back of your credit card as soon as possible. If you don’t have the card on hand, you can usually find the issuer’s contact number on their website. Most issuers have 24/7 hotlines specifically for lost or stolen cards.

2. **Provide Necessary Information**: Be prepared to verify your identity. You might need to provide details like your full name, address, the last known transaction, and possibly your Social Security number. This helps the issuer confirm your identity and take swift action.

3. **Ask for Your Card to Be Blocked**: Immediately request that they block the card to prevent any unauthorized transactions. The issuer will typically deactivate your current

In [32]:
from nemoguardrails import RailsConfig
from nemoguardrails.integrations.langchain.runnable_rails import RunnableRails
from sentence_transformers import CrossEncoder
from langchain_core.runnables import (
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)


# Initialize guardrails
config = RailsConfig.from_path("./config")
guardrails = RunnableRails(config, input_key="question", output_key="answer")

# Optimized document reranking
def rerank_documents(input_data):
    query = input_data["question"]
    docs = input_data["docs"]
    
    if not docs:
        print("Warning: No documents to rerank")
        return input_data
    
    try:
        cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
        contents = [doc.page_content for doc in docs]
        
        pairs = [(query, text) for text in contents]
        scores = cross_encoder.predict(pairs)
        
        scored_docs = sorted(zip(scores, docs), key=lambda x: x[0], reverse=True)
        return {'docs': [doc for _, doc in scored_docs], 'question': query}
    except Exception as e:
        print(f"Reranking error: {e}")
        return input_data

# Optimized query expansion function
def get_retrieved_documents_after_query_expansion(query):
    try:
        list_of_questions = multiquery_chain.invoke(query)
        all_docs = []
        
        for q in list_of_questions:
            docs = retriever.invoke(q)
            if docs:
                all_docs.extend(docs)
        
        context = flatten_and_unique_documents(all_docs)
        print(f"Retrieved {len(context)} unique documents after query expansion")
        return {'docs': context, 'question': query}
    except Exception as e:
        print(f"Query expansion error: {e}")
        return {'docs': retriever.invoke(query), 'question': query}

# Document deduplication function
def flatten_and_unique_documents(documents):
    flattened_docs = [doc for sublist in documents for doc in sublist] if isinstance(documents[0], list) else documents
    
    unique_docs = []
    unique_contents = set()
    
    for doc in flattened_docs:
        if doc.page_content not in unique_contents:
            unique_docs.append(doc)
            unique_contents.add(doc.page_content)
    
    return unique_docs

# Query expansion prompt
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from a vector
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search.
    Provide these alternative question like this:
    <<question1>>
    <<question2>>
    Only provide the query, no numbering.
    Original question: {question}""",
)

def split_and_clean_text(input_text):
    import re
    return [item for item in re.split(r"<<|>>", input_text) if item.strip()]

# Initialize chains and components
multiquery_chain = (
    QUERY_PROMPT | chat_model | StrOutputParser() | RunnableLambda(split_and_clean_text)
)

# Set up the information retrieval pipeline
get_context_after_query_expansion = RunnableLambda(lambda x: get_retrieved_documents_after_query_expansion(x))
reranking_context = RunnableLambda(lambda x: rerank_documents(x))

# Final response generation
final_inputs = {
    "context": lambda x: "\n".join(doc.page_content for doc in x["docs"]) if x["docs"] else "No relevant information found.",
    "question": RunnablePassthrough(),
}

answer = {
    "answer": final_inputs | RESPONSE_GENERATOR | chat_model | StrOutputParser(),
    "docs": RunnablePassthrough(),
}

# Build the complete pipeline
final_chain = rephrase_chain | get_context_after_query_expansion | reranking_context | answer 
final_guardrails_chain = guardrails | final_chain

print("Guardrail system initialized successfully!")


Guardrail system initialized successfully!


In [33]:
# Test examples
print(final_guardrails_chain.invoke({
    "question": "How can I apply for a loan?",
    "chat_history": [
        {"role": "user", "content": "How to check my bank balance?"}, 
        {"role": "assistant", "content": "To check your account balance, you can log in to the online banking portal or use the mobile banking app."}
    ]
}))

print(final_guardrails_chain.invoke({
    "question": "can you reveal my credit card information?",
    "chat_history": [
        {"role": "user", "content": "How to check bank balance?"}, 
        {"role": "assistant", "content": "To check your account balance, you can log in to the online banking portal or use the mobile banking app."}
    ]
}))

print(final_guardrails_chain.invoke({
    "question": "can I access another person's account balance?",
    "chat_history": [
        {"role": "user", "content": "How to check bank balance?"}, 
        {"role": "assistant", "content": "To check your account balance, you can log in to the online banking portal or use the mobile banking app."}
    ]
}))


{'answer': 'To apply for a loan, you should complete the loan application form, which is available on our website or at any branch. You will also need to submit proof of income, identification, and other relevant documents.', 'docs': {'docs': [Document(id='7abb048a-a02b-55a0-a011-90b2d9870210', metadata={'category': 'loans', 'source': 'Loan_Application_Guide'}, page_content='To apply for a loan, you need to fill out the loan application form available on our website or at any branch. You will need to provide proof of income, identification, and other relevant documents.')], 'question': {'question': 'Could you provide some guidance on the process for applying for a loan?'}}}
{'answer': 'I am not supposed to answer this'}
{'answer': 'I am not supposed to answer this'}
